In [ ]:
import zipfile

# Specify the path to your ZIP file
zip_file_path = '/content/archive.zip'  # Change this to the path of your ZIP file

# Specify the directory where you want to extract the contents
extracted_dir_path = '/content/Imagedataset'  # Change this to your desired extraction directory

# Create the extraction directory if it doesn't exist
import os
os.makedirs(extracted_dir_path, exist_ok=True)

# Open the ZIP file and extract its contents
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir_path)

# List the extracted files and directories
extracted_files = os.listdir(extracted_dir_path)
print("Extracted files and directories:")
for item in extracted_files:
    print(item)


Extracted files and directories:
cropped_images


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/content/Imagedataset'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/content/Imagedataset/cropped_images/mark_ruffalo/mark_ruffalo56.png
/content/Imagedataset/cropped_images/mark_ruffalo/mark_ruffalo7.png
/content/Imagedataset/cropped_images/mark_ruffalo/mark_ruffalo20.png
/content/Imagedataset/cropped_images/mark_ruffalo/mark_ruffalo43.png
/content/Imagedataset/cropped_images/mark_ruffalo/mark_ruffalo22.png
/content/Imagedataset/cropped_images/mark_ruffalo/mark_ruffalo38.png
/content/Imagedataset/cropped_images/mark_ruffalo/mark_ruffalo3.png
/content/Imagedataset/cropped_images/mark_ruffalo/mark_ruffalo28.png
/content/Imagedataset/cropped_images/mark_ruffalo/mark_ruffalo62.png
/content/Imagedataset/cropped_images/mark_ruffalo/mark_ruffalo36.png
/content/Imagedataset/cropped_images/mark_ruffalo/mark_ruffalo16.png
/content/Imagedataset/cropped_images/mark_ruffalo/mark_ruffalo2.png
/content/Imagedataset/cropped_images/mark_ruffalo/mark_ruffalo4.png
/content/Imagedataset/cropped_images/mark_ruffalo/mark_ruffalo59.png
/content/Imagedataset/cropped_images/m

DATA SPLIT-TRAIN AND TEST SETS

In [ ]:
import os
import random
import shutil

data_folder = '/content/Imagedataset/cropped_images/'
train_folder = 'train'
test_folder = 'test'

split_ratio = 0.8  # Training set will contain 80% of the data
os.makedirs(train_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)

for class_folder in os.listdir(data_folder):
    class_path = os.path.join(data_folder, class_folder)
    if not os.path.isdir(class_path):
        continue  # Skip non-folder entries

    train_class_folder = os.path.join(train_folder, class_folder)
    test_class_folder = os.path.join(test_folder, class_folder)
    os.makedirs(train_class_folder, exist_ok=True)
    os.makedirs(test_class_folder, exist_ok=True)

    for image_file in os.listdir(class_path):
        image_path = os.path.join(class_path, image_file)

        if random.random() < split_ratio:
            shutil.copy2(image_path, train_class_folder)
        else:
            shutil.copy2(image_path, test_class_folder)


 Importing Libraries

In [ ]:
# hide warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras.applications.densenet import DenseNet169

from keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from keras.layers import Dense, Conv2D, MaxPooling2D, ZeroPadding2D, Flatten, GlobalAveragePooling2D, InputLayer, Dropout, SpatialDropout2D, BatchNormalization, Resizing, Rescaling, RandomFlip, RandomRotation

Fine tuning

In [ ]:
dense_net = DenseNet169(weights='imagenet', include_top=False, input_shape=None)

for layer in dense_net.layers:
    layer.trainable = False

Getting train , test , validate Images from our dirs

In [ ]:
train_data = tf.keras.utils.image_dataset_from_directory(
    train_folder,
    image_size=(200, 200),
    batch_size=64)

valid_data = tf.keras.utils.image_dataset_from_directory(
    test_folder,
    image_size=(200, 200),
    batch_size=64)

Found 219 files belonging to 5 classes.
Found 55 files belonging to 5 classes.


Building the model

In [ ]:
model = Sequential()
model.add(dense_net)
model.add(GlobalAveragePooling2D())
model.add(Dense(256 , activation ='relu'))
model.add(BatchNormalization())
model.add(Dense(128 , activation ='relu'))
model.add(BatchNormalization())
model.add(Dense(5, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet169 (Functional)    (None, None, None, 1664)  12642880  
                                                                 
 global_average_pooling2d (G  (None, 1664)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 256)               426240    
                                                                 
 batch_normalization (BatchN  (None, 256)              1024      
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 batch_normalization_1 (Batc  (None, 128)              5

train the model

In [ ]:
model_history = model.fit(train_data, epochs=20, validation_data=valid_data, verbose=1)

Epoch 1/20
4/4 [==============================] - 60s 15s/step - loss: 0.4626 - accuracy: 0.8995 - val_loss: 3.2273 - val_accuracy: 0.3273
Epoch 2/20
4/4 [==============================] - 60s 16s/step - loss: 0.3274 - accuracy: 0.9498 - val_loss: 2.7922 - val_accuracy: 0.3091
Epoch 3/20
4/4 [==============================] - 53s 13s/step - loss: 0.2422 - accuracy: 0.9680 - val_loss: 2.3980 - val_accuracy: 0.3273
Epoch 4/20
4/4 [==============================] - 68s 19s/step - loss: 0.1788 - accuracy: 0.9817 - val_loss: 2.0212 - val_accuracy: 0.4182
Epoch 5/20
4/4 [==============================] - 51s 13s/step - loss: 0.1365 - accuracy: 0.9909 - val_loss: 1.8698 - val_accuracy: 0.4364
Epoch 6/20
4/4 [==============================] - 50s 13s/step - loss: 0.1121 - accuracy: 0.9954 - val_loss: 1.9497 - val_accuracy: 0.4364
Epoch 7/20
4/4 [==============================] - 60s 16s/step - loss: 0.0690 - accuracy: 1.0000 - val_loss: 1.9478 - val_accuracy: 0.4364
Epoch 8/20
4/4 [===========

Save the model

In [ ]:
model.save('cnn_dense.h5')

# Use The Mode With Video or Camera

to access camera

In [ ]:
from IPython.display import display, Javascript

def enable_camera():
    js = Javascript('''
        let video;
        let stream;

        async function enableCamera() {
            stream = await navigator.mediaDevices.getUserMedia({ 'video': true });
            video = document.createElement('video');
            document.body.appendChild(video);
            video.srcObject = stream;
            await video.play();
        }

        enableCamera();

        document.addEventListener('keydown', (event) => {
            console.log('Key pressed: ' + event.key);  // Add this line for debugging
            if (event.key === 'o') {
                console.log('Turning off camera');  // Add this line for debugging
                if (stream) {
                    const tracks = stream.getTracks();
                    tracks.forEach(track => track.stop());
                    video.remove();
                }
            }
        });
    ''')
    display(js)

enable_camera()



<IPython.core.display.Javascript object>

In [ ]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')


In [ ]:
import base64
import cv2
import numpy as np
from PIL import Image
from IPython.display import display, Image

from keras.models import load_model
from keras.preprocessing import image

model = load_model('cnn_dense.h5')

model_dict = {0: "chris_evans", 1: "chris_hemsworth", 2: "mark_ruffalo", 3: "robert_downey_jr", 4: "scarlett_johansson"}

# Use the camera index that works for your system
camera_index = 0  # You can try different values like 1 if 0 doesn't work

video_capture = cv2.VideoCapture(camera_index)

if not video_capture.isOpened():
    print("Error: Could not open video capture.")
    exit()

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def face_extractor(img):
    faces = face_cascade.detectMultiScale(img, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    if len(faces) == 0:
        return None

    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 255), 2)
        cropped_face = img[y:y + h, x:x + w]
    return cropped_face

while True:
    ret, frame = video_capture.read()

    if not ret:
        print("Error: Could not read a frame from the camera.")
        break

    face = face_extractor(frame)
    if type(face) is np.ndarray:
        face = cv2.resize(face, (180, 180))
        im = Image.fromarray(cv2.cvtColor(face, cv2.COLOR_BGR2RGB))
        img_array = np.array(im)
        img_array = np.expand_dims(img_array, axis=0)
        pred = model.predict(img_array)[0]
        maxindex = int(np.argmax(pred))
        print(maxindex)

        name = "None matching"
        print(pred[maxindex])
        if pred[maxindex] > 0.4:
            name = model_dict[maxindex]
        cv2.putText(frame, name, (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)

        _, frame_buffer = cv2.imencode('.jpg', frame)
        if not frame_buffer.empty():
            img_base64 = base64.b64encode(frame_buffer).decode('utf-8')
            display(Image(data=f"data:image/jpeg;base64,{img_base64}"))
    else:
        cv2.putText(frame, "No face found", (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)

        _, frame_buffer = cv2.imencode('.jpg', frame)
        if not frame_buffer.empty():
            img_base64 = base64.b64encode(frame_buffer).decode('utf-8')
            display(Image(data=f"data:image/jpeg;base64,{img_base64}"))

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()


Error: Could not open video capture.
Error: Could not read a frame from the camera.
